In [4]:
%load_ext pycodestyle_magic

In [5]:
%flake8_on

In [6]:
from IOHexperimenter import IOH_function, IOH_logger
import datetime
from ea import EA
from bfgs import BFGS
from mlsl import MLSL
import handover

In [9]:
def switch(func, budget, alg1, alg2, tau, target):
    """ Switch function to switch between two pre-defined algorithms.

    Parameters:
    -------------
    func : object
        The function that will be optimized.

    budget : int
        Budget for function evaluations, total.

    a1 : string
        The algorithm that should run first.
        Options: 'MLSL', 'BFGS', 'CMAES', 'EA', 'PSO'.

    a2 : string
        The algorithm that should run second.
        Options: 'MLSL', 'BFGS', 'CMAES', 'EA', 'PSO'.

    tau : float
        The function value at which algorithms should switch from A1 to A2.

    target : float
        The fixed-target value until which A2 should optimize

    """
    # Initialize A1
    if alg1 == 'MLSL':
        a1 = MLSL((budget/2), 'a1')
    elif alg1 == 'BFGS':
        a1 = BFGS((budget/2), 'a1')
    elif alg1 == 'EA':
        a1 = EA(((budget/2), 'a1'))
    else:
        print(f'The algorithm {alg1} is unknown.')

    if alg2 == 'MLSL':
        a2 = MLSL((budget/2), 'a2')
    elif alg2 == 'BFGS':
        a2 = BFGS((budget/2), 'a2')
    elif alg2 == 'EA':
        a2 = EA(((budget/2), 'a2'))
    else:
        print(f'The algorithm {alg2} is unknown.')

    # Run A1
    a1(func, tau, target)
    print(f' switch_precision so far: {func.best_so_far_precision}')

    # Handover of information
    handover.mlsl_bfgs(a1, a2)

    # Run A2
    a2(func, tau, target)
    print(f' switch_precision so far: {func.best_so_far_precision}')

In [ ]:
path = "/Users/dschroeder/Documents/Master Computer Science/Master_Thesis/code_and_data/experiments/"
now = datetime.datetime.now()
experiment_ID = str(now.day) + "-" + str(now.month) + "-" + str(now.strftime('%H')) + str(now.strftime('%M')) + str(now.strftime('%S'))
experiment_name =  path + "simple_switch/experiment" + experiment_ID

instances = [1, 2, 3, 4, 5]

for instance in instances:
    f12 = IOH_function(12, 20, instance, suite = 'BBOB')
    l = IOH_logger(experiment_name, "run", "simple_switch_tau_30", "switch-mlsl-bfgs")
    f12.add_logger(l)
    for run in range(0, 5):
        print(f' run {run} instance {instance}')
        if run > 0:
            f12.reset()
        switch(f12, 10000, 'MLSL', 'BFGS', 30, 1e-8)
    f12.clear_logger()